# 2D Convolutional Variational Autoencoder

*   Input data shape (12906, 658, 5) with one-hot encoding.
*   The training accuracy with 100 epochs can lead to 97%. The following training accuracy with 40 epochs is up to 96.5%.



In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
## adjust the length of DNA sequence with 658
## remove the uncorrect characters 'K','M', 'N', 'R', 'S', 'W', 'Y'
df = pd.read_csv (r'/content/train_features.csv')
def equal_length(seq):
    seq2 = seq.ljust(1058, '-') 
    fir = seq2[0:658]
    remove_characters = ['K','M', 'N', 'R', 'S', 'W', 'Y']
    for character in remove_characters:
        fir = fir.replace(character, "-")
    return fir
dna = df.dna.apply(equal_length)
## Example for arbitery DNA sequence
dna[3]

'AACATTATATTTTATTTTTGGTGCATGAGCTGGAATAGTAGGAACTTCATTAAGTATGCTAATTCGAGCTGAATTAGGAAACCCAGGTGCATTAATTGGAGATGACCAAATTTATAATGTTATTGTTACTGCACATGCATTTATTATGATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGAAATTGATTAGTTCCTTTAATATTAGGAGCCCCAGATATAGCTTTTCCTCGAATAAATAATATAAGTTTTTGATTACTTCCTCCTTCATTAACGCTCCTCTTAATAAGAAGTCTAGTG---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------'

In [3]:

from numpy import argmax

def one_hot_encode(seq):
  # define universe of possible input values
  #alphabet = 'ACGT-KMNRSWY'
  alphabet = 'ACGT-'
  # define a mapping of chars to integers
  char_to_int = dict((c, i) for i, c in enumerate(alphabet))
  int_to_char = dict((i, c) for i, c in enumerate(alphabet))
  # integer encode input data
  integer_encoded = [char_to_int[char] for char in seq]
  # one hot encode
  onehot_encoded = list()
  for value in integer_encoded:
	  letter = [0 for _ in range(len(alphabet))]
	  letter[value] = 1
	  onehot_encoded.append(letter)
  return(np.asarray(onehot_encoded))
dna_code = dna.apply(one_hot_encode)

In [6]:
x_dna = dna_code.to_list()
x_dna = np.array(x_dna)
y_train = pd.read_csv (r'/content/train_labels.csv')
ytrain = np.array(y_train['labels'], dtype = "uint32")
x_dna.shape

(12906, 658, 5)

In [7]:
xtrain = np.expand_dims(x_dna, -1)
model = tf.keras.models.Sequential([  
  keras.Input(shape=(658,5,1)),
  layers.Conv2D(64, 3, activation="relu", strides=(4, 1),padding="same"),
  layers.Conv2D(32, 3, activation="relu",  strides=(2, 2),padding="same"),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dropout(0.2, seed=3),
  layers.Dense(1220, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(xtrain, ytrain, epochs=40, validation_split=0.2, verbose=2)

Epoch 1/40
323/323 - 34s - loss: 6.1627 - accuracy: 0.0889 - val_loss: 4.4352 - val_accuracy: 0.2754
Epoch 2/40
323/323 - 2s - loss: 2.2141 - accuracy: 0.5899 - val_loss: 0.9393 - val_accuracy: 0.8284
Epoch 3/40
323/323 - 2s - loss: 0.5904 - accuracy: 0.8714 - val_loss: 0.4637 - val_accuracy: 0.9206
Epoch 4/40
323/323 - 2s - loss: 0.3390 - accuracy: 0.9178 - val_loss: 0.4177 - val_accuracy: 0.9345
Epoch 5/40
323/323 - 2s - loss: 0.2329 - accuracy: 0.9359 - val_loss: 0.3899 - val_accuracy: 0.9450
Epoch 6/40
323/323 - 2s - loss: 0.1829 - accuracy: 0.9500 - val_loss: 0.3940 - val_accuracy: 0.9469
Epoch 7/40
323/323 - 2s - loss: 0.1389 - accuracy: 0.9616 - val_loss: 0.3668 - val_accuracy: 0.9500
Epoch 8/40
323/323 - 2s - loss: 0.1177 - accuracy: 0.9671 - val_loss: 0.4125 - val_accuracy: 0.9531
Epoch 9/40
323/323 - 2s - loss: 0.1054 - accuracy: 0.9681 - val_loss: 0.3894 - val_accuracy: 0.9558
Epoch 10/40
323/323 - 2s - loss: 0.0975 - accuracy: 0.9718 - val_loss: 0.4676 - val_accuracy: 0.955